In [1]:
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
from ricerca import get_ordered_words, get_ordered_words_2
from def_overlap_funcs import filter_words

## Ricerca onomasiologica
Questo tipo di ricerca va a trovare una parola partendo dalle sue definizioni. Per fare l'esercizio viene usato il file definito in aula, nella tabella sotto sono riportate tutte le definizioni tokenizzate e pulite da stop words e punteggiatura.

In [2]:
data = pd.read_csv('TLN-definitions-23.csv', sep=';')
data = data.applymap(lambda s: filter_words(set(word_tokenize(s))))

data

,door,ladybug,pain,blurriness
0,"{used, closing, two, construction, rooms, pass...","{spots, red, flying, black, small, insect, typ...","{physical, feeling, mental, distress}","{focus, sight}"
1,"{closed, opened, opening}","{red, dots, black, luck, insect, good, many, c...","{physical, bad, feeling, emotional, hurt}","{absence, shapelessness, borders, definite}"
2,"{door, room, two, let, get, object, wall, open...","{red, dots, black, orange, insect, color, fly}","{felling, beings, experience, conscious, suffer}","{felt, sensation, see, ca, border, something, ..."
3,"{one, usable, another, area, access}","{red, insect, back, small}","{concept, living, suffering, describes}","{lack, sharpness}"
4,"{allows, area, access, delimits, structure}","{insect, flying, small, round}","{physical, feeling, discomfort}","{clarity, lack, characteristic, precision}"
5,"{pass, used, block, passage, object, moved}","{tendentially, flying, red, small, beatle, ins...","{sensation, unpleasant, burning}","{eyes, blurred, blurring, sensation, see, ever..."
6,"{consists, made, rotates, historically, used, ...","{red, dots, black, used, small, luck, bring, i...","{physical, felt, creatures, sensation, damaged...","{physical, accurately, whether, certain, kind,..."
7,"{separate, wall, used, go, rooms, opened, clos...","{red, dots, black, culture, small, luck, insec...","{physical, injury, caused, pain, feeling, emot...","{memory, image, lack, bad, vision, unfocused, ..."
8,"{another, opened, place, access, something, or...","{red, dots, black, insect, small}","{sentiment, sorrow, expresses}","{condition, seeing, totally, makes, bright, cl..."
9,"{access, room}","{red, small, dotted, coat, insect, yellow}","{physical, exertion, resulting, sensation, emo...","{confusion, physical, exertion, state, resulti..."


### Lista genus
Vado a calcolare una lista di parole comuni per ognuno dei quattro termini. Questo mi permette di adottare la **strategia del genus**, ovvero far partire la ricerca del target da una parola molto usata nella sua definizione.
Vengono considerate solo le definizioni che hanno più di 4 parole. In realtà la lista dei genus non viene modificata molto

In [3]:
r = data.apply(lambda column: Counter(p for i in column for p in i if len(i) > 4).most_common())
r

door          [(object, 12), (room, 8), (access, 8), (allows...
ladybug       [(insect, 24), (red, 23), (black, 20), (small,...
pain          [(physical, 9), (emotional, 7), (feeling, 6), ...
blurriness    [(see, 7), (something, 5), (image, 5), (visual...
dtype: object

### Estrazione definizioni
Creo una lista con tutte le parole delle definizioni per un dato target che voglio trovare. La lista delle parole è già pulita da punteggiatura e stop words e tutte le parole compaiono una sola volta.

In [4]:
words_list_door = list(set().union(*data['door']))
words_list_bug = list(set().union(*data['ladybug']))
words_list_pain = list(set().union(*data['pain']))
words_list_blur = list(set().union(*data['blurriness']))

### Ricerca
Il metodo prende la lista di parole e il punteggio di similarità con la lista delle definizioni.
Il metodo prende tutti i synset di un genus e cicla su di essi, andando a calcolare la similarità tra il contesto del senso e il contesto delle definizioni. Ripete questa operazione su tutti i genus della lista in input.
Alla fine ordina la lista di $(senso,\ score\ similarità)$ risultato in modo *non crescente* per valore di similarità.

Nel metodo **get_ordered_words_2** l'intuizione è quella di non limitarsi agli iponimi del genus, ma provare a scendere anche lungo i figli. Viene mantenuto un limite per portare l'algoritmo a una terminazione veloce.
Per sfoltire si considerano solo i figli che hanno un valore di similarità associato sopra una certa soglia (la metà del valore massimo).
Per i **termini concreti** si è notato un miglioramento nella soluzione, in quanto i tra i sensi più alti ci sono gli elementi target delle descrizioni.
Per i **termini astratti** non ci sono stati cambiamenti degni di nota.
Questo approccio andrebbe rifinito per avere un *criterio d'arresto* più generico.

In una prima versione del metodo di base si provavano a prendere anche in considerazione i meronimi.

#### DOOR

In [5]:
get_ordered_words(r['door'], words_list_door)[0:10]

[('doorway', 9),
 ('screen_door', 6),
 ('cabin', 6),
 ('leak', 5),
 ('strap_hinge', 5),
 ('saltbox', 5),
 ('solar_house', 5),
 ('compartment', 4),
 ('study', 4),
 ('door', 4)]

In [6]:
get_ordered_words_2(r['door'], words_list_door, 4)[0:10]

[('doorway', 9),
 ('screen_door', 6),
 ('cabin', 6),
 ('door', 6),
 ('leak', 5),
 ('strap_hinge', 5),
 ('saltbox', 5),
 ('solar_house', 5),
 ('wedge', 5),
 ('compartment', 4)]

#### LADYBUG

In [7]:
get_ordered_words(r['ladybug'], words_list_bug)[0:10]

[('reddish_orange', 4),
 ('calendula', 4),
 ('dipterous_insect', 3),
 ('leaf_miner', 3),
 ('lepidopterous_insect', 3),
 ('psocopterous_insect', 3),
 ('scarlet', 3),
 ('horn_fly', 3),
 ('housefly', 3),
 ('streamer_fly', 3)]

In [8]:
get_ordered_words_2(r['ladybug'], words_list_bug, 4)[0:10]

[('reddish_orange', 4),
 ('calendula', 4),
 ('ladybug', 4),
 ('wax_begonia', 4),
 ('oncidium', 4),
 ('dipterous_insect', 3),
 ('leaf_miner', 3),
 ('lepidopterous_insect', 3),
 ('psocopterous_insect', 3),
 ('scarlet', 3)]

#### PAIN

In [9]:
get_ordered_words(r['pain'], words_list_pain)[0:10]

[('constriction', 5),
 ('pinch', 4),
 ('concussion', 4),
 ('wrong', 4),
 ('photalgia', 4),
 ('referred_pain', 4),
 ('sting', 4),
 ('vision', 3),
 ('brain_damage', 3),
 ('strain', 3)]

In [10]:
get_ordered_words_2(r['pain'], words_list_pain, 4)[0:10]

[('constriction', 5),
 ('pinch', 4),
 ('concussion', 4),
 ('wrong', 4),
 ('photalgia', 4),
 ('referred_pain', 4),
 ('sting', 4),
 ('vision', 3),
 ('brain_damage', 3),
 ('strain', 3)]

#### BLURRINESS

In [11]:
get_ordered_words(r['blurriness'], words_list_blur)[0:10]

[('vision', 5),
 ('like', 5),
 ('look', 5),
 ('collage', 4),
 ('acuity', 4),
 ('naked_eye', 4),
 ('quality', 4),
 ('iconography', 3),
 ('likeness', 3),
 ('binocular_vision', 3)]

In [12]:
get_ordered_words_2(r['blurriness'], words_list_blur, 4)[0:10]

[('vision', 5),
 ('like', 5),
 ('look', 5),
 ('collage', 4),
 ('acuity', 4),
 ('naked_eye', 4),
 ('quality', 4),
 ('iconography', 3),
 ('likeness', 3),
 ('binocular_vision', 3)]